In [2]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [3]:
# Carregando DATASET
arquivo = "../data/heart-disease.csv"

# Lendo o arquivo
dataset = pd.read_csv(arquivo, delimiter=',')

# Exibe primeiras linhas do dataset
dataset.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [4]:
# Separação em conjunto de treino e conjunto de teste com holdout
test_size = 0.20 # amanho do conjunto de teste
seed = 7 # semente aleatória

# Separação em conjuntos de treino e testes
array = dataset.values
X = array[:,0:13]
y = array[:,13]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, shuffle=True, random_state=seed, stratify=y) # holdout com stratificação

# parâmetros e partições da validação cruzada
scoring = 'accuracy'
num_particoes = 10
kfold = StratifiedKFold(n_splits=num_particoes, shuffle=True, random_state=seed) # validação cruzada com estratificação

IndexError: index 14 is out of bounds for axis 1 with size 14

In [10]:
# title Criação e validação dos modelos

np.random.seed(7) # semente global

# lista para armezenar os modelos
models = [('KNN', KNeighborsClassifier()), ('CART', DecisionTreeClassifier()), ('NB', GaussianNB()), ('SVM', SVC())]

# criando modelos e adicionando a lista

# Listas para armazenar os resultados
results = []
names = []

# Avaliação dos modelos

for name, model in models:
  cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
  results.append(cv_results)
  names.append(name)
  msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
  print(msg)

  #Boxplot de comparação dos modelos
  fig = plt.figure(figsize=(15, 20))
  fig.suptitle('Comparação dos modelos')
  ax = fig.add_subplot(111)
  plt.boxplot(results)
  ax.set_xticklabels(names)
  plt.show()

NameError: name 'X_train' is not defined

In [11]:
# Criação e avaliação de modelos: dados padronizados e normalizados

#Listas para armazenar os pipelines e os resultados para todas as visões do dataset
pipelines = []
results = []
names = []

# Criando elementos do pipelines
#Algoritmos que serão utilizados
knn = ('KNN', KNeighborsClassifier())
cart = ('CART', DecisionTreeClassifier())
naive_bayes = ('NB', GaussianNB())
svm = ('SVM', SVC())

#Transformações que serão utilizadas
standard_scaler = ('StandardScaler', StandardScaler()) # padronização
min_max_scaler = ('MinMaxScaler', MinMaxScaler()) # normalização

#Montando os pipelines

#Dataset original
pipelines.append(('KNN-orig', Pipeline([knn])))
pipelines.append(('CART-orig', Pipeline([cart])))
pipelines.append(('NB-orig', Pipeline([naive_bayes])))
pipelines.append(('SVM-orig', Pipeline([svm])))

#Dataset Padronizado
pipelines.append(('KNN-padr', Pipeline([standard_scaler, knn])))
pipelines.append(('CART-padr', Pipeline([standard_scaler, cart])))
pipelines.append(('NB-padr', Pipeline([standard_scaler, naive_bayes])))
pipelines.append(('SVM-padr', Pipeline([standard_scaler, svm])))

#Dataset Normalizado
pipelines.append(('KNN-norm', Pipeline([min_max_scaler, knn])))
pipelines.append(('CART-norm', Pipeline([min_max_scaler, cart])))
pipelines.append(('NB-norm', Pipeline([min_max_scaler, naive_bayes])))
pipelines.append(('SVM-norm', Pipeline([min_max_scaler, svm])))

# Executando os pipelines
for name, model in pipelines:
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %.3f (%.3f)" % (name, cv_results.mean(), cv_results.std()) # formatando para 3 casas decimais
    print(msg)

# Boxplot de comparação dos modelos
fig = plt.figure(figsize=(25,6))
fig.suptitle('Comparação dos Modelos - Dataset orginal, padronizado e normalizado')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names, rotation=90)
plt.show()

NameError: name 'X_train' is not defined

In [21]:
#title Otimização dos hiperparâmetros

# SVM
np.random.seed(7)

# Definição de components do pipelines
KNN = ('SVM', SVC())
standard_scaler = ('StandarScaler', StandardScaler())
min_max_scaler = ('MinMaxScaler', MinMaxScaler())

pipelines = []
pipelines.append(('knn-orig', Pipeline(steps=[knn])))
pipelines.append(('knn-padr', Pipeline(steps=[standard_scaler, knn])))
pipelines.append(('knn-norm', Pipeline(steps=[min_max_scaler, knn])))

param_grid = {
    'KNN__n_neighbors': [3, 5, 7, 9],
    'KNN__metric': ["euclidean", "manhattan", "minkowski"],
}


# prepara e executa o GridSearchCV
for name, pipeline in pipelines:
  grid_search = GridSearchCV(pipeline, param_grid, cv=5)
  grid_search.fit(X_train, y_train)

  #imprime melhor configuração
  print(f"Melhores parâmetros para {name}: {grid_search.best_params_}")
  print(f"Melhor acurácia para {name}: {grid_search.best_score_}")

Melhores parâmetros para knn-orig: {'KNN__metric': 'manhattan', 'KNN__n_neighbors': 3}
Melhor acurácia para knn-orig: 0.6941326530612245
Melhores parâmetros para knn-padr: {'KNN__metric': 'manhattan', 'KNN__n_neighbors': 7}
Melhor acurácia para knn-padr: 0.8552721088435373
Melhores parâmetros para knn-norm: {'KNN__metric': 'euclidean', 'KNN__n_neighbors': 3}
Melhor acurácia para knn-norm: 0.8430272108843537


In [12]:
pipeline = Pipeline(steps=[('StandarScaler', StandardScaler()),('KNN', KNeighborsClassifier())])
pipeline.fit(X_train, y_train)
predictions = pipeline.predict(X_test)
print(accuracy_score(y_test, predictions))

NameError: name 'X_train' is not defined

In [28]:
#avaliação do modelo com o conjunto de teste

# preparando o modelo
scaler = StandardScaler().fit(X_train) # ajuste do scaler com o conjunto de treino
rescaledX = scaler.transform(X_train) # aplicação da padronização no conjunto de treino
model = KNeighborsClassifier()
model.fit(rescaledX, y_train)

# Estimativa de acuracia no conjunto de teste
rescaledTestX = scaler.transform(X_test) # aplicação da padronização no conjunto de teste
predictions = model.predict(rescaledTestX)
print(accuracy_score(y_test, predictions))

0.819672131147541


In [29]:
#title Preparando o modelo com o dataset inteiro
scaler = StandardScaler().fit(X)
rescaledX = scaler.transform(X)
model.fit(rescaledX, y)

KNeighborsClassifier()

In [30]:
#title Simulando a classificação do modelo em dados não vistos

data = dataset

atributos = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']
entrada = pd.DataFrame(data, columns=atributos)

array_entrada = entrada.values
x_entrada = array_entrada[:,0:13].astype(float)

# padronização nos dados de entrada usando o scaler utilizado em x
rescaledEntradaX = scaler.transform(x_entrada)
print(rescaledEntradaX)


[[ 9.52196602e-01  6.81005225e-01  1.97312292e+00 ...  1.46399175e+00
  -7.14428871e-01 -3.78482354e+00]
 [-1.91531289e+00  6.81005225e-01  1.00257707e+00 ...  3.41455020e+00
  -7.14428871e-01 -3.78482354e+00]
 [-1.47415758e+00 -1.46841752e+00  3.20312162e-02 ...  1.07291444e-03
   1.24459328e+00 -3.78482354e+00]
 ...
 [ 1.50364073e+00  6.81005225e-01 -9.38514634e-01 ...  3.25200366e+00
   2.65082205e-01 -5.12921878e-01]
 [ 2.90463642e-01  6.81005225e-01 -9.38514634e-01 ... -3.24020160e-01
   2.65082205e-01 -2.14887271e+00]
 [ 2.90463642e-01 -1.46841752e+00  3.20312162e-02 ... -2.27457861e+00
   2.65082205e-01 -2.14887271e+00]]


In [34]:
# Predição de classes dos dados de entrada
saidas = model.predict(rescaledEntradaX)
print(saidas)

[1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0.
 0. 1. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 1.
 1. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 1. 0. 1. 0. 1. 1. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0. 1.
 0. 0. 1. 1. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0.
 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1.]


#### Exportar modelo, scaler e pipeline com pickle

In [35]:
import pickle
pickle_out = open('classificador.pkl', 'wb')
pickle.dump(model, pickle_out)
pickle_out.close()

In [38]:
pipeline_out = open('pipelines.pkl', 'wb')
pickle.dump(pipeline, pipeline_out)
pipeline_out.close()

In [39]:
scaler_out = open('scaler.pkl', 'wb')
pickle.dump(scaler, scaler_out)
scaler_out.close()

In [40]:
# Salvar dados csv de teste
X_test_df = pd.DataFrame(X_test, columns=dataset.columns[:-1])
y_test_df = pd.DataFrame(y_test, columns=[dataset.columns[-1]])
X_test_df.to_csv("../data/" + 'x_test_heart_disease.csv', index=False)
y_test_df.to_csv("../data/" + 'y_test_heart_disease.csv', index=False)